In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

house = pd.read_csv("train.csv")
houseTest = pd.read_csv("test.csv")

Id = houseTest["id"]
houseTest = houseTest.drop(["id"],axis=1)


# house
print("-"*50+ "train简单查看统计学信息" +"-"*50)
print(house.info())
print("-"*50+ "test简单查看统计学信息" +"-"*50)
print(houseTest.info())

print("-"*50 + "train查看缺失值" + "-"*50)
print(house.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(houseTest.isnull().sum().sort_values(ascending=False).head(10))



--------------------------------------------------train简单查看统计学信息--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196539 entries, 0 to 196538
Data columns (total 19 columns):
时间          196539 non-null int64
小区名         196539 non-null int64
小区房屋出租数量    195538 non-null float64
楼层          196539 non-null int64
总楼层         196539 non-null float64
房屋面积        196539 non-null float64
房屋朝向        196539 non-null object
居住状态        20138 non-null float64
卧室数量        196539 non-null int64
厅的数量        196539 non-null int64
卫的数量        196539 non-null int64
出租方式        24230 non-null float64
区           196508 non-null float64
位置          196508 non-null float64
地铁线路        91778 non-null float64
地铁站点        91778 non-null float64
距离          91778 non-null float64
装修情况        18492 non-null float64
月租金         196539 non-null float64
dtypes: float64(12), int64(6), object(1)
memory usage: 28.5+ MB
None
----------------------------------------

In [2]:
house["位置"][house["位置"].isnull()] = house["位置"].dropna().mode().values
house["区"][house["区"].isnull()] = house["区"].dropna().mode().values
house["小区房屋出租数量"][house["小区房屋出租数量"].isnull()] = house["小区房屋出租数量"].dropna().mode().values
houseTest["位置"][houseTest["位置"].isnull()] = houseTest["位置"].dropna().mode().values
houseTest["区"][houseTest["区"].isnull()] = houseTest["区"].dropna().mode().values
houseTest["小区房屋出租数量"][houseTest["小区房屋出租数量"].isnull()] = houseTest["小区房屋出租数量"].dropna().mode().values

print("-"*50 + "train查看缺失值" + "-"*50)
print(house.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(houseTest.isnull().sum().sort_values(ascending=False).head(10))



--------------------------------------------------train查看缺失值--------------------------------------------------
装修情况        178047
居住状态        176401
出租方式        172309
距离          104761
地铁站点        104761
地铁线路        104761
月租金              0
房屋面积             0
小区名              0
小区房屋出租数量         0
dtype: int64
--------------------------------------------------test查看缺失值--------------------------------------------------
装修情况        52072
居住状态        51796
出租方式        51308
地铁站点        29785
地铁线路        29785
距离          29785
房屋面积            0
小区名             0
小区房屋出租数量        0
楼层              0
dtype: int64


In [3]:
# 装修情况肯定对租金价格有较大影响，我们现在有两万的装修数据，可以尝试下预测
# 居住状态不太好预测，这个属于人为因素，但是应该有人居住的房子一定租金较低
# 出租方式，妥妥的有影响，中介收费不一，朋友介绍肯定便宜
# 距离应该是地铁站点到房子的距离，交通的影响
# 地铁站这个不太好预测，因为住房附近可能压根没地铁
# 小区出租数量，位置，区 可以直接干掉


In [4]:
# 特征工程

# 房屋朝向
print("-"*50+"house房屋朝向"+"-"*50)
house_orientation_unique = house["房屋朝向"].unique()
print(house["房屋朝向"].unique())
# print(house["房屋朝向"].value_counts())
print("-"*50+"houseTest房屋朝向"+"-"*50)
houseTest_orientation_unique = houseTest["房屋朝向"].unique()
print(houseTest["房屋朝向"].unique())
# print(houseTest["房屋朝向"].value_counts())
print()
print("house 有 houseTest没有:")
print ([i for i in house_orientation_unique if i not in houseTest_orientation_unique])
print("house 没有 houseTest有:")
print ([i for i in houseTest_orientation_unique if i not in house_orientation_unique])
print("house 有 houseTest有:")
print ([i for i in houseTest_orientation_unique if i in house_orientation_unique])

house_orientation = pd.DataFrame({'东':[0]*196539,'南':[0]*196539,
                                 '西':[0]*196539,'北':[0]*196539,
                                 '东南':[0]*196539,'东北':[0]*196539,
                                 '西南':[0]*196539,'西北':[0]*196539,})
houseTest_orientation = pd.DataFrame({'东':[0]*196539,'南':[0]*196539,
                                 '西':[0]*196539,'北':[0]*196539,
                                 '东南':[0]*196539,'东北':[0]*196539,
                                 '西南':[0]*196539,'西北':[0]*196539,})
house = house.join(house_orientation)
# house

# 将房屋朝向整成八个特征
def orientation_dong(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    dong = "东"
    if dong in lst:
        return 1
    else:
        return 0

def orientation_nan(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    nan = "南"
    if nan in lst:
        return 1
    else:
        return 0

    
def orientation_xi(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    xi = "西"
    if xi in lst:
        return 1
    else:
        return 0


def orientation_bei(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    bei = "北"
    if bei in lst:
        return 1
    else:
        return 0

    
def orientation_dongNan(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    dongNan = "东南"
    if dongNan in lst:
        return 1
    else:
        return 0

    
def orientation_dongBei(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    dongBei = "东北"
    if dongBei in lst:
        return 1
    else:
        return 0

    
def orientation_xiNan(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    xiNan = "西南"
    if xiNan in lst:
        return 1
    else:
        return 0

    
def orientation_xiBei(x):
    lst = []
    for i in x.split(" "):
        lst.append(i)
    xiBei = "西北"
    if xiBei in lst:
        return 1
    else:
        return 0

    
house['东'] = house['房屋朝向'].apply(lambda x : orientation_dong(x) )
house['南'] = house['房屋朝向'].apply(lambda x : orientation_nan(x) )
house['西'] = house['房屋朝向'].apply(lambda x : orientation_xi(x) )
house['北'] = house['房屋朝向'].apply(lambda x : orientation_bei(x) )
house['东南'] = house['房屋朝向'].apply(lambda x : orientation_dongNan(x) )
house['东北'] = house['房屋朝向'].apply(lambda x : orientation_dongBei(x) )
house['西南'] = house['房屋朝向'].apply(lambda x : orientation_xiNan(x) )
house['西北'] = house['房屋朝向'].apply(lambda x : orientation_xiBei(x) )

houseTest['东'] = houseTest['房屋朝向'].apply(lambda x : orientation_dong(x) )
houseTest['南'] = houseTest['房屋朝向'].apply(lambda x : orientation_nan(x) )
houseTest['西'] = houseTest['房屋朝向'].apply(lambda x : orientation_xi(x) )
houseTest['北'] = houseTest['房屋朝向'].apply(lambda x : orientation_bei(x) )
houseTest['东南'] = houseTest['房屋朝向'].apply(lambda x : orientation_dongNan(x) )
houseTest['东北'] = houseTest['房屋朝向'].apply(lambda x : orientation_dongBei(x) )
houseTest['西南'] = houseTest['房屋朝向'].apply(lambda x : orientation_xiNan(x) )
houseTest['西北'] = houseTest['房屋朝向'].apply(lambda x : orientation_xiBei(x) )

house = house.drop("房屋朝向",axis=1)
houseTest = houseTest.drop("房屋朝向",axis=1)
house


--------------------------------------------------house房屋朝向--------------------------------------------------
['东南' '东' '南' '东北' '西南' '南 北' '西北' '西' '北' '东南 西南 西' '东 南' '东 东南' '南 西南'
 '东南 西北' '东 西南' '东南 南 西南' '东 南 西' '西南 西北' '东 东北' '东南 南' '东 西' '南 东' '西南 西'
 '东南 西南' '南 西' '西南 北' '西 北' '北 东北' '西 西北' '东南 东北' '西北 东北' '西北 北' '西南 东北'
 '东 东南 南' '东南 西' '南 西北' '东 北' '南 东北' '东 南 北' '东 西 北' '东 东南 北' '西南 西 东北'
 '东南 南 北' '东 南 西 北' '东 西北' '东南 北' '东南 西南 西北' '东南 南 西南 西' '南 西 北' '南 西南 西'
 '东 西 东北' '东 东南 西南' '南 西南 北' '东 南 西北 北' '东 西北 北' '北 南' '西 西北 北'
 '东 东南 南 西南 西' '北 西' '南 北 东北' '东南 西 北' '东 西南 北' '东 南 西南' '南 西南 西 西北']
--------------------------------------------------houseTest房屋朝向--------------------------------------------------
['东' '东南' '南' '西北' '西南' '西' '东北' '北' '南 北' '东 南' '东 北' '南 西' '东 西' '东 东南'
 '东 西南 北' '南 西南' '东 西北' '西北 北 东北' '东南 南' '东南 南 北' '西南 北' '东南 东北' '东南 西南'
 '东 西南' '东南 西北' '南 西南 北' '南 东' '西南 西北' '西北 北' '东 南 北' '南 西南 西 西北' '南 西北'
 '东 东北' '西 北' '东 南 西 北' '西南 西' '西北 东北' '东南 北' '东南 南 西南'

,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,居住状态,卧室数量,厅的数量,卫的数量,...,装修情况,月租金,东,南,西,北,东南,东北,西南,西北
0,1,3072,0.128906,2,0.236364,0.008628,NaN,1,1,1,...,NaN,5.602716,0,0,0,0,1,0,0,0
1,1,3152,0.132812,1,0.381818,0.017046,NaN,1,0,0,...,NaN,16.977929,1,0,0,0,0,0,0,0
2,1,5575,0.042969,0,0.290909,0.010593,NaN,2,1,2,...,NaN,8.998302,0,0,0,0,1,0,0,0
3,1,3103,0.085938,2,0.581818,0.019199,NaN,3,2,2,...,NaN,5.602716,0,1,0,0,0,0,0,0
4,1,5182,0.214844,0,0.545455,0.010427,NaN,2,1,1,...,NaN,7.300509,0,0,0,0,0,1,0,0
5,1,1192,0.039062,2,0.309091,0.012579,NaN,2,1,1,...,NaN,4.923599,0,1,0,0,0,0,0,0
6,1,1122,0.125000,0,0.381818,0.010593,NaN,3,1,1,...,NaN,6.621392,0,1,0,0,0,0,0,0
7,1,1251,0.128906,2,0.363636,0.018040,NaN,4,2,2,...,NaN,14.091681,0,1,0,0,0,0,0,0
8,1,4718,0.246094,2,0.309091,0.007850,NaN,1,1,1,...,NaN,4.584041,0,0,0,0,0,0,1,0
9,1,2654,0.218750,2,0.890909,0.020026,NaN,2,1,2,...,NaN,39.558574,0,0,0,0,1,0,0,0


In [5]:
print("-"*50 + "train查看缺失值" + "-"*50)
print(house.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(houseTest.isnull().sum().sort_values(ascending=False).head(10))


--------------------------------------------------train查看缺失值--------------------------------------------------
装修情况        178047
居住状态        176401
出租方式        172309
距离          104761
地铁站点        104761
地铁线路        104761
西北               0
小区名              0
小区房屋出租数量         0
楼层               0
dtype: int64
--------------------------------------------------test查看缺失值--------------------------------------------------
装修情况        52072
居住状态        51796
出租方式        51308
距离          29785
地铁站点        29785
地铁线路        29785
西北              0
卫的数量            0
小区名             0
小区房屋出租数量        0
dtype: int64


In [6]:
# house['装修情况'] = house['装修情况'].fillna(house['装修情况'].mean())
# house['居住状态'] = house['居住状态'].fillna(house['居住状态'].mean())
# house['出租方式'] = house['出租方式'].fillna(house['出租方式'].mean())
# house['距离'] = house['距离'].fillna(0)
# house['地铁站点'] = house['地铁站点'].fillna(0)
# house['地铁线路'] = house['地铁线路'].fillna(0)

# houseTest['装修情况'] = houseTest['装修情况'].fillna(houseTest['装修情况'].mean())
# houseTest['居住状态'] = houseTest['居住状态'].fillna(houseTest['居住状态'].mean())
# houseTest['出租方式'] = houseTest['出租方式'].fillna(houseTest['出租方式'].mean())
# houseTest['距离'] = houseTest['距离'].fillna(0)
# houseTest['地铁站点'] = houseTest['地铁站点'].fillna(0)
# houseTest['地铁线路'] = houseTest['地铁线路'].fillna(0)


In [7]:
from sklearn.model_selection import train_test_split

x = house.drop(["月租金"],axis=1)
y = house.月租金
x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.5, random_state=10)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn import model_selection

x = house.drop(["月租金"],axis=1)
y = house.月租金

x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=10)

clf_est =XGBRegressor(random_state=11)
clf_param_grid = {'n_estimators': [500,1000,1500],
                  'learning_rate': [0.01,0.05],
                 'max_depth': [6,7,8], 
                  'min_child_weight': [3,4,5], 
                  'gamma':[0.7,0.9], 
                  'subsample':[0.8,0.7,0.6],
                  'colsample_bytree':[0.15,0.25,0.35]}
clf_grid = model_selection.GridSearchCV(clf_est, clf_param_grid, n_jobs=2, cv=5, verbose=1)
clf = clf_grid
clf.fit(x_train, y_train)
print('BestParams:' + str(clf_grid.best_params_))
# clf = XGBRegressor().fit(x_train,y_train)


from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf,x_test,y_test)
print("Boost: ",scores.mean())

# scores_cb = cross_val_score(cb,x_test,y_test)
# print("CatBoost: ",scores_cb.mean())

predictions = clf.predict(houseTest.astype(float))
Submission = pd.DataFrame({'id': Id, 
                           'price': predictions})
Submission.to_csv('Submission.csv',index=False,sep=',')
print("I LOVE YOUU")

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Fitting 5 folds for each of 972 candidates, totalling 4860 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 28.6min


In [ ]:
# from sklearn import metrics

# y_pred_xgb = xgb.predict(x_test)
# # 均方根误差
# print(metrics.mean_squared_error(y_test,y_pred_xgb))